In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        29/10/2025  -  23:00:15           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    06/11/2025  -  23:55:40           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#

# Competición Perretes y Gatos

## Iteración 5 - Fine-tuning VGG16

**Situación actual**: Iteración 5 con Transfer Learning consiguió 0.86380 (posición #7).

**Pasos que voy a hacer con Fine-tuning**:
- VGG16 pre-entrenado en ImageNet (14M imágenes)
- Descongelar las últimas 4 capas convolucionales (block5)
- Learning rate MUY bajo (1e-5) para no romper pesos pre-entrenados
- Solo 10 épocas adicionales con ajuste fino
- Objetivo: +2-4% score → 0.88-0.90

**kaggle score**: 0.89552, lo cual mejora respecto al anterior -> siguiente planteamiento aplicar esemble con 3 modelos

In [ ]:
# Importo las librerías necesarias para trabajar con el modelo y los datos
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras

# Establezco una semilla aleatoria para que los resultados sean reproducibles
seed = 42
keras.utils.set_random_seed(seed)

# Rutas de datos - cambia solo el nombre del dataset si es diferente
DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Versión de Keras:", keras.__version__)
print("Rutas configuradas correctamente")

## Carga y Preparación de los Datos

Cargo las imágenes del conjunto de entrenamiento y las divido automáticamente en dos partes:
- **80% para entrenamiento**: El modelo aprende de estas imágenes
- **20% para validación**: Evalúo el rendimiento en cada época sin que el modelo las haya visto

Esta división me permite detectar si el modelo está memorizando (overfitting) en lugar de
aprender patrones generalizables. Todas las imágenes se redimensionan a **224x224 píxeles**,
que es el tamaño de entrada estándar requerido por VGG16.

El modo `binary` es crucial para clasificación de 2 clases, ya que genera etiquetas 0/1
en lugar de vectores one-hot, lo que simplifica la salida y evita problemas de calibración.

In [ ]:
# Tamaño de imagen - 224x224 para VGG16
image_size = (224, 224)

# Batch size
batch_size = 125

# Cargo las imágenes del directorio de entrenamiento
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary",  # Cambio a binary
)

print(f"Tamaño del conjunto de entrenamiento: {len(train_ds)} batches")
print(f"Tamaño del conjunto de validación: {len(val_ds)} batches")
print(f"Total imágenes de entrenamiento aproximadas: {len(train_ds) * batch_size}")
print(f"Total imágenes de validación aproximadas: {len(val_ds) * batch_size}")

## Data Augmentation (Aumento de Datos)

Aplico transformaciones aleatorias a las imágenes **solo durante el entrenamiento** para
hacer el modelo más robusto y evitar que memorice. Estas transformaciones simulan variaciones
naturales que puede encontrar el modelo en datos reales:

- **RandomFlip horizontal**: Voltea la imagen (perros/gatos pueden estar orientados de cualquier forma)
- **RandomRotation (±36°)**: Rotaciones suaves que no distorsionan la imagen
- **RandomZoom (±10%)**: Acerca o aleja la imagen ligeramente
- **RandomTranslation (±10%)**: Desplaza la imagen horizontal y verticalmente

He configurado transformaciones **conservadoras** (valores bajos) porque VGG16 ya conoce
características generales de millones de imágenes. No necesito augmentation agresivo como
en un modelo entrenado desde cero, solo variaciones sutiles para mejorar la generalización.

In [ ]:
# Secuencia de transformaciones aleatorias aplicadas solo durante entrenamiento
# Más conservador para Transfer Learning
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),  # ±36° suficiente
    keras.layers.RandomZoom(0.1),      # ±10% sin distorsión
    keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
], name="data_augmentation")

print("Data Augmentation configurado (modo conservador para Transfer Learning)")

## Construcción del Modelo - Transfer Learning

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.applications import VGG16

# Forma de entrada: 224x224 con 3 canales RGB (requerido por VGG16)
input_shape = image_size + (3,)

# Cargo VGG16 pre-entrenado sin la parte superior (sin clasificación)
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape,
    pooling='avg'  # Global Average Pooling
)

# congelo todas las capas de VGG16 - no las voy a entrenar
base_model.trainable = False

# Construyo el modelo completo
model = Sequential([
    keras.Input(shape=input_shape),
    data_augmentation,  # Augmentation solo en entrenamiento
    base_model,         # VGG16 congelado
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Salida binaria
], name='VGG16_Transfer_Learning')

print(f"Modelo Transfer Learning creado")
print(f"Capas VGG16 congeladas: {len(base_model.layers)}")
print(f"Solo entrenaremos: Dense(256) + Dropout + Dense(1)")
model.summary()

## Compilación y Entrenamiento - Transfer Learning

In [ ]:
%%time

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # LR bajo para Transfer Learning
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

epochs = 15  # Transfer Learning converge rápido

# ReduceLROnPlateau: reduce el learning rate si se estanca
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

print("Configuración Transfer Learning VGG16:")
print(f"Épocas: {epochs}")
print(f"Optimizer: Adam (lr=0.0001)")
print(f"Modelo base: VGG16 ImageNet (congelado)")
print(f"Entrenamos: Solo 2 capas Dense")
print("-" * 60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[reduce_lr],
    verbose=1
)

print("-" * 60)
print(f"Entrenamiento completado en {len(history.history['loss'])} épocas")
print(f"Val Accuracy final: {history.history['val_accuracy'][-1]:.4f}")
print(f"Val Precision final: {history.history['val_precision'][-1]:.4f}")
print(f"Val Recall final: {history.history['val_recall'][-1]:.4f}")

## Visualización de las Curvas de Aprendizaje

In [ ]:
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))

# Gráfico de pérdida
plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:, "loss"], lw=2, label='Pérdida en entrenamiento')
plt.plot(logs.loc[1:, "val_loss"], lw=2, label='Pérdida en validación')
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Evolución de la Pérdida")
plt.legend()
plt.grid(True, alpha=0.3)

# Gráfico de precisión
plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:, "accuracy"], lw=2, label='Precisión en entrenamiento')
plt.plot(logs.loc[1:, "val_accuracy"], lw=2, label='Precisión en validación')
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.title("Evolución de la Precisión")
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("RESULTADOS FINALES:")
print("="*60)
print(f"Precisión final en entrenamiento: {logs['accuracy'].iloc[-1]:.4f}")
print(f"Precisión final en validación: {logs['val_accuracy'].iloc[-1]:.4f}")
print(f"Pérdida final en entrenamiento: {logs['loss'].iloc[-1]:.4f}")
print(f"Pérdida final en validación: {logs['val_loss'].iloc[-1]:.4f}")
print("="*60)

## Guardado del Modelo

In [ ]:
model.save("model.keras")
print("Modelo guardado como 'model.keras'")

## Evaluación con Datos Suplementarios

In [ ]:
supplementary_ds = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary",  # Binary consistente
)

print("Evaluando con datos suplementarios...")
print("-" * 60)

results = model.evaluate(supplementary_ds, return_dict=True, verbose=1)

print("-" * 60)
print("\nRESULTADOS EN DATOS SUPLEMENTARIOS:")
print(f"Precisión: {results['accuracy']:.4f}")
print(f"Pérdida: {results['loss']:.4f}")
print("-" * 60)

## Generación de Predicciones para Kaggle

In [ ]:
%%time

predictions_dict = {}

print(f"Generando predicciones para {len(os.listdir(TEST_PATH))} imágenes del conjunto de test...")
print("Usando umbral 0.5 para clasificación binaria (probabilidad >= 0.5 → Dog, < 0.5 → Cat)")
print("-" * 70)

# Itero sobre todas las imágenes del directorio de test
for img in os.listdir(TEST_PATH):
    img_path = os.path.join(TEST_PATH, img)
    file_name = img_path.split('/')[-1]
    file_no_extension = file_name.split('.')[0]  # ID numérico de la imagen
    
    # Cargo la imagen y la preparo para el modelo
    img_loaded = keras.utils.load_img(img_path, target_size=image_size)
    img_array = keras.utils.img_to_array(img_loaded)
    img_array = keras.ops.expand_dims(img_array, 0)  # Añado dimensión de batch
    
    # Predigo probabilidad con el modelo fine-tuned
    # Salida sigmoid: valor entre 0 (Cat) y 1 (Dog)
    prediction = model.predict(img_array, verbose=0)[0][0]
    
    # Conversión explícita a clase binaria
    label = 1 if prediction >= 0.5 else 0
    
    predictions_dict[int(file_no_extension)] = label

print("-" * 70)
print(f"Predicciones completadas: {len(predictions_dict)} imágenes procesadas")

## Creación del Archivo de Submission para Kaggle

In [ ]:
# Creo el DataFrame con las predicciones y lo ordeno por ID
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"])
submission = submission.sort_values(by='id', ascending=True)

# Guardo el archivo CSV para Kaggle
submission.to_csv('submission.csv', index=False)

print("="*70)
print("ARCHIVO DE SUBMISSION CREADO: submission.csv")
print("="*70)

# Estadísticas de las predicciones
print("\nDISTRIBUCION DE PREDICCIONES:")
print("-" * 70)
print(submission["label"].value_counts())
print()
print(f"Clase 0 (Cat): {(submission['label'] == 0).sum():4d} imágenes ({100*(submission['label'] == 0).sum()/len(submission):5.1f}%)")
print(f"Clase 1 (Dog): {(submission['label'] == 1).sum():4d} imágenes ({100*(submission['label'] == 1).sum()/len(submission):5.1f}%)")
print(f"Total:         {len(submission):4d} imágenes")

# Muestro ejemplos de predicciones
print("\nPRIMERAS 10 PREDICCIONES:")
print(submission.head(10).to_string(index=False))

print("\nULTIMAS 10 PREDICCIONES:")
print(submission.tail(10).to_string(index=False))

# detectar si todas las predicciones son de una sola clase
print("\n" + "="*70)
if (submission['label'] == 0).sum() == len(submission) or (submission['label'] == 1).sum() == len(submission):
    print("ALERTA: TODAS LAS PREDICCIONES SON DE UNA SOLA CLASE")
    print("El modelo está completamente sesgado. NO enviar a Kaggle.")
    print("="*70)
else:
    print("VERIFICACION EXITOSA")
    print("  - Distribucion de clases balanceada")
    print("  - Archivo listo para enviar a Kaggle")
    print("  - Mejora esperada respecto a Iter 5: +2-4% (0.88-0.90)")
    print("="*70)